#Application Pratique sur le dataset SQUAD 2.0

In [ ]:
!pip install transformers datasets accelerate -q

In [ ]:
from transformers import SqueezeBertForQuestionAnswering, AutoTokenizer
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import torch

In [ ]:
datasets = load_dataset("squad_v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
batch_size = 16
max_length = 384  # The maximum length of a feature (question and context)
doc_stride = 128  # The allowed overlap between two part of the context when splitting is performed.

In [ ]:
pad_on_right = tokenizer.padding_side == "right"

In [ ]:
def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    examples["question"] = [q.lstrip() for q in examples["question"]]

    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",  # Tronquer seulement le contexte, pas la question
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (
                offsets[token_start_index][0] <= start_char
                and offsets[token_end_index][1] >= end_char
            ):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while (
                    token_start_index < len(offsets)
                    and offsets[token_start_index][0] <= start_char
                ):
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
push_to_hub_model_id = "squeezebert-uncased-finetuned-squad-v2"

from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model = SqueezeBertForQuestionAnswering.from_pretrained("squeezebert/squeezebert-uncased")
tokenizer = AutoTokenizer.from_pretrained("squeezebert/squeezebert-uncased")

# Utilisez la fonction de prétraitement sur vos données
tokenized_datasets = datasets.map(
    prepare_train_features, batched=True, remove_columns=datasets["train"].column_names
)

warmup_steps = len(tokenized_datasets['train']) / batch_size * 1 * 0.08

training_args = TrainingArguments(
    output_dir=push_to_hub_model_id,
    push_to_hub=True,
    num_train_epochs=1,
    warmup_steps=warmup_steps,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    gradient_accumulation_steps=4,  # Accumule les gradients pour simuler un plus grand batch size
    per_device_train_batch_size=batch_size,  # Réduit pour économiser la mémoire GPU
    per_device_eval_batch_size=batch_size,  # Réduit pour économiser la mémoire GPU
    fp16=True,  # Activez si votre GPU supporte FP16 pour une formation plus rapide et moins gourmande en mémoire
    # gradient_checkpointing=True,  # Active le checkpointing pour économiser la mémoire
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
)

trainer.train()

Some weights of SqueezeBertForQuestionAnswering were not initialized from the model checkpoint at squeezebert/squeezebert-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
100,5.792700
200,4.742000
300,3.806700
400,3.366500
500,2.903300
600,2.496900
700,2.142300
800,1.916600
900,1.736800
1000,1.623900


TrainOutput(global_step=2058, training_loss=2.198796045212519, metrics={'train_runtime': 1481.4605, 'train_samples_per_second': 88.935, 'train_steps_per_second': 1.389, 'total_flos': 8271091649544192.0, 'train_loss': 2.198796045212519, 'epoch': 1.0})

In [ ]:
contexts = [
    "The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ('Norman' comes from 'Norseman') raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.",
    "The further decline of Byzantine state-of-affairs paved the road to a third attack in 1185, when a large Norman army invaded Dyrrachium, owing to the betrayal of high Byzantine officials. Some time later, Dyrrachium—one of the most important naval bases of the Adriatic—fell again to Byzantine hands.",
    "Many locals and tourists frequent the southern California coast for its popular beaches, and the desert city of Palm Springs is popular for its resort feel and nearby open spaces."
]

questions = [
    "In what country is Normandy located?",
    "When did the Normans attack Dyrrachium?",
    "Which region of California is Palm Springs located in?"
]

In [ ]:
import torch

model.eval()

for question, context in zip(questions, contexts):
    # Tokenize the input question-context pair
    inputs = tokenizer.encode_plus(question, context, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

    # Send inputs to the same device as your model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        # Forward pass, get model outputs
        outputs = model(**inputs)

    # Extract the start and end positions of the answer in the tokens
    answer_start_scores, answer_end_scores = outputs.start_logits, outputs.end_logits
    answer_start_index = torch.argmax(answer_start_scores)  # Most likely start of answer
    answer_end_index = torch.argmax(answer_end_scores) + 1  # Most likely end of answer; +1 for inclusive slicing

    # Convert token indices to the actual answer text
    answer_tokens = inputs['input_ids'][0, answer_start_index:answer_end_index]
    answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)

    print(f"Question: {question}")
    print(f"Predicted answer: {answer}\n")

In [ ]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/204M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Samuel-DD/squeezebert-uncased-finetuned-squad-v2/commit/f27e08803440f799d36fe824c89fed4ee29b8cc0', commit_message='End of training', commit_description='', oid='f27e08803440f799d36fe824c89fed4ee29b8cc0', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

trainer.save_model("/content/drive/MyDrive/MyModel/" + "squeezebert/squeezebert-uncased")  # Sauvegarde le modèle au chemin spécifié
tokenizer.save_pretrained("/content/drive/MyDrive/MyModel" + "squeezebert/squeezebert-uncased")  # Sauvegarde le tokenizer au même chemi

('/content/drive/MyDrive/MyModelsqueezebert/squeezebert-uncased/tokenizer_config.json',
 '/content/drive/MyDrive/MyModelsqueezebert/squeezebert-uncased/special_tokens_map.json',
 '/content/drive/MyDrive/MyModelsqueezebert/squeezebert-uncased/vocab.txt',
 '/content/drive/MyDrive/MyModelsqueezebert/squeezebert-uncased/added_tokens.json',
 '/content/drive/MyDrive/MyModelsqueezebert/squeezebert-uncased/tokenizer.json')